In [1]:
import yaml
import MetaTrader5 as mt5
import pandas as pd
import os
import numpy as np

In [2]:
INITIAL_CASH = 10_000

# Portfolio equity curve

In [3]:
def get_portfolio_equity_curve(equity_curves, initial_equity, date_range):
    total = pd.DataFrame()

    for name, curve in equity_curves.items():
        eq = curve.copy()
        eq = eq.reset_index().rename(columns={'index':'Date'})[['Date','Equity']].sort_values(by='Date')
        eq['Date'] = pd.to_datetime(eq['Date'])
        eq['Date'] = eq['Date'].dt.floor('D').dt.date

        eq = eq.groupby('Date').agg({'Equity':'last'})

        eq = eq.reindex(date_range)
        
        eq.Equity = eq.Equity.ffill()
        eq.Equity = eq.Equity.fillna(initial_equity)
    
        eq['variacion'] = eq['Equity'] - eq['Equity'].shift(1)
        eq['variacion_porcentual'] = eq['variacion'] / eq['Equity'].shift(1)
        
        df_variacion = pd.DataFrame(
            {
                f'variacion_{name}': eq.variacion_porcentual.fillna(0)
            }
        )
        
        total = pd.concat([total, df_variacion], axis=1)

    total = total.reset_index().rename(columns={'index':'Date'})

    # Inicializa el valor de equity
    total['Equity'] = initial_equity

    # Lista de columnas con las variaciones porcentuales
    variation_cols = [col for col in total.columns if col.startswith('variacion')]

    # Calcular la curva de equity
    for i in range(1, len(total)):
        previous_equity = total.loc[i-1, 'Equity']  # Equity del periodo anterior
        
        # Calcula el impacto monetario de cada bot por separado y suma el resultado
        impact_sum = 0
        for col in variation_cols:
            variation = total.loc[i, col]
            impact_sum += previous_equity * variation
        
        # Actualiza el equity sumando el impacto monetario total
        total.loc[i, 'Equity'] = previous_equity + impact_sum

    # Resultado final

    total = total.set_index('Date')
    return total[['Equity']]

# FTMO Simulator

In [4]:
def ftmo_simulator(equity_curve, initial_cash):
    equity_curve['month'] = pd.to_datetime(equity_curve.index)
    equity_curve['month'] = pd.to_datetime(equity_curve['month'], errors='coerce')  # Asegúrate de que sea datetime
    equity_curve['month'] = equity_curve['month'].dt.to_period('M')  # Convertir a un periodo mensual
    
    equity_curve.fillna(0, inplace=True)

    # Identificar índices de los valores máximo y mínimo por mes
    max_indices = equity_curve.groupby('month')['Equity'].idxmax()
    min_indices = equity_curve.groupby('month')['Equity'].idxmin()

    # Combinar índices únicos
    unique_indices = pd.Index(max_indices).union(pd.Index(min_indices))

    equity_curve = equity_curve.loc[unique_indices]
    
    # Inicializar acumuladores globales
    total_positive_hits = 0
    total_negative_hits = 0
    all_time_to_positive = []
    all_time_to_negative = []

    # Simulación para cada mes como punto de partida
    for i in range(0, len(equity_curve), 2):
        perc_change = 0
        time_to_positive = []
        time_to_negative = []
        
        actual_equity = equity_curve.iloc[i].Equity
        
        months_elapsed = 0

        # Iterar desde el mes de inicio hacia adelante
        for j in range(i, len(equity_curve)):
            
            future_equity = equity_curve.iloc[j].Equity
            
            if i == 0 and j == 0:
                perc_change = ((future_equity - initial_cash) / initial_cash) * 100
                
            else:
                perc_change = ((future_equity - actual_equity) / actual_equity) * 100
            
            months_elapsed += 0.5

            if perc_change >= 10:
                total_positive_hits += 1
                time_to_positive.append(months_elapsed)
                months_elapsed = 0
                break

            elif perc_change <= -10:
                total_negative_hits += 1
                time_to_negative.append(months_elapsed)
                months_elapsed = 0
                break

        # Guardar tiempos de esta simulación
        all_time_to_positive.extend(time_to_positive)
        all_time_to_negative.extend(time_to_negative)

    return total_positive_hits, total_negative_hits, all_time_to_positive, all_time_to_negative


# Margin metrics

In [5]:
def calculate_margin_metrics(all_trades, portfolio_equity_curve):
    # Convertir columnas a datetime
    for df in all_trades.values():
        df["EntryTime"] = pd.to_datetime(df["EntryTime"])
        df["ExitTime"] = pd.to_datetime(df["ExitTime"])

    # Concatenar y calcular los eventos
    all_events = pd.concat([
        pd.concat([
            df[["EntryTime", "margin"]].rename(columns={"EntryTime": "time", "margin": "change"}).round(3),
            df[["ExitTime", "margin"]].rename(columns={"ExitTime": "time", "margin": "change"}).assign(change=lambda x: -x["change"]).round(3)
        ]) for df in all_trades.values()
    ])

    # Ordenar por tiempo
    all_events = all_events.sort_values(['time', 'change'], ascending=[True, False]).reset_index(drop=True)

    # Calcular el margen acumulado
    all_events["margin"] = all_events["change"].cumsum()


    all_events['time'] = pd.to_datetime(all_events['time']).dt.date
    all_events.set_index('time', inplace=True)

    all_events = pd.merge(
        all_events,
        portfolio_equity_curve,
        left_index=True,
        right_index=True,
        how='left'
    )

    all_events = all_events.round(2)

    all_events['margin_level'] = ((all_events['Equity'] / all_events['margin']) * 100)

    all_events['free_margin'] = (all_events['Equity'] - all_events['margin'])

    stop_outs = all_events[(all_events['margin_level'] < 50) & (all_events['margin_level'] != -1*np.inf)]
    margin_calls = all_events[(all_events['margin_level'] < 100) & (all_events['margin_level'] != -1*np.inf)]
    
    return all_events, margin_calls, stop_outs

In [6]:
def max_drawdown(equity_curve, verbose=True):
    # Calcular el running max de la equity curve
    running_max = np.maximum.accumulate(equity_curve)
    
    # Calcular el drawdown
    drawdown = (equity_curve - running_max) / running_max
    
    # Encontrar el valor máximo de drawdown y la fecha correspondiente
    max_drawdown_value = np.min(drawdown) * 100  # Convertir el drawdown a porcentaje
    max_drawdown_date = equity_curve.index[np.argmin(drawdown)]
    
    if verbose:
        print(f"Máximo drawdown: {max_drawdown_value:.2f}%")
        print(f"Fecha del máximo drawdown: {max_drawdown_date}")

    return max_drawdown_value

## Obtener trades y curvas de equity originales

In [7]:
import MetaTrader5 as mt5

# Inicializar MetaTrader 5
if not mt5.initialize():
    print("Error al inicializar MT5")
    quit()

pd.set_option('display.max_columns', 500) # number of columns to be displayed


if not mt5.initialize():
    print("initialize() failed, error code =",mt5.last_error())
    quit()

root = './backbone/data'

with open('configs/live_trading.yml', 'r') as file:
    strategies = yaml.safe_load(file)

with open("./configs/leverages.yml", "r") as file_name:
    leverages = yaml.safe_load(file_name)

timeframes_to_number = {
    'H1': 16385,
    'H2': 16386,
    'H3': 16387,
    'H4': 16388,
}

all_equity_curves = {}
all_trades = {}

for strategy_name, configs in strategies.items():
    
    instruments_info = configs['instruments_info']
    wfo_params = configs['wfo_params']
    opt_params = configs['opt_params']
    name = strategy_name.split('.')[-1]
    use_wfo = wfo_params['use_wfo']
    
    root =  os.path.join('./backtesting_pipeline/strategies', name)
    
    folder = ''
    
    if use_wfo:
        folder = 'full_analysis'
    
    else:
        folder = 'preliminar_analysis'
    
    dir = os.path.join(root, folder)
    
    for ticker, info in instruments_info.items():
        symbol_info = mt5.symbol_info(ticker)

        leverage = leverages[ticker]
        
        timeframe = timeframes_to_number[info['timeframe']]
        key = f'{ticker}_{timeframe}'
        
        full_key = f'{name}_{ticker}_{timeframe}'
        
        # if full_key in candidates:
        equity = pd.read_csv(
            os.path.join(dir, key, 'equity.csv'), index_col=0
        )
        equity.index = pd.to_datetime(equity.index)
        
        trades = pd.read_csv(
            os.path.join(dir, key, 'trades.csv'), usecols=['Size', 'EntryPrice', 'ExitPrice', 'PnL', 'EntryTime', 'ExitTime']
        )
        trades.EntryTime = pd.to_datetime(trades.EntryTime)
        trades.ExitTime = pd.to_datetime(trades.ExitTime)
        
        trades['margin'] = (np.abs(trades['Size']) * trades['EntryPrice']) / leverage
        
        all_equity_curves[full_key] = equity
        all_trades[full_key] = trades

In [8]:
import pandas as pd

min_date = None
max_date = None

for name, curve in all_equity_curves.items():
    # Convertir las fechas a UTC si son tz-naive
    actual_date = curve.index[0].tz_localize('UTC') if curve.index[0].tz is None else curve.index[0].tz_convert('UTC')
    
    # Si min_date es None, inicializar con la primera fecha
    if min_date is None:
        min_date = actual_date
    # Comparar si la fecha actual es menor que min_date
    elif actual_date < min_date:
        min_date = actual_date

    # Si max_date es None, inicializar con la última fecha
    curve_last_date = curve.index[-1].tz_localize('UTC') if curve.index[-1].tz is None else curve.index[-1].tz_convert('UTC')
    
    if max_date is None:
        max_date = curve_last_date
    # Comparar si la fecha actual es mayor que max_date
    elif curve_last_date > max_date:
        max_date = curve_last_date

# Mostrar las fechas encontradas
print(f"Min Date: {min_date}")
print(f"Max Date: {max_date}")

# Calcular min_date y max_date
min_date = min_date.date()
max_date = max_date.date()

print(min_date)
print(max_date)

date_range = pd.to_datetime(pd.date_range(start=min_date, end=max_date, freq='D'))
print(date_range)

Min Date: 2021-01-04 00:00:00+00:00
Max Date: 2024-11-01 00:00:00+00:00
2021-01-04
2024-11-01
DatetimeIndex(['2021-01-04', '2021-01-05', '2021-01-06', '2021-01-07',
               '2021-01-08', '2021-01-09', '2021-01-10', '2021-01-11',
               '2021-01-12', '2021-01-13',
               ...
               '2024-10-23', '2024-10-24', '2024-10-25', '2024-10-26',
               '2024-10-27', '2024-10-28', '2024-10-29', '2024-10-30',
               '2024-10-31', '2024-11-01'],
              dtype='datetime64[ns]', length=1398, freq='D')


# Obtener riesgos simulados

In [9]:
risks = [0.25, 0.5, 1, 1.5, 2]

not_run = {
    'NVDA':[1, 1.5, 2],
    'BABA':[1, 1.5, 2],
    'US2000.cash':[1, 1.5, 2],
    'EURUSD':[0.25, 0.5],
    'USDCHF':[0.25, 0.5],
    'GBPUSD':[0.25, 0.5],
}

original_and_syntetic_trades = {}
original_and_syntetic_eq_curves = {}

for strategy in all_trades.keys():
    ticker = strategy.split('_')[1] # <-- Creo que esto podria hacerse mejor
    
    symbol_info = mt5.symbol_info(ticker)
    leverage = leverages[ticker]
    
    print(strategy)
    original_and_syntetic_trades[f'{strategy}'] = {}
    original_and_syntetic_eq_curves[f'{strategy}'] = {}
    
    for risk in risks:
        
        if ticker in not_run.keys():
            if risk in not_run[ticker]:
                print(f'no correr {ticker} con risk {risk}')
                continue
            
        trades = all_trades[strategy]
        eq_curve = all_equity_curves[strategy]
        
        # ADVERTENCIA esto ya no es necesario pq se calcula internamente
        trades = pd.merge(
            trades,
            eq_curve['Equity'],
            left_on='ExitTime',
            right_index=True,
            how='inner'
        )

        # calculo el porcentaje de retorno
        
        trades['ReturnPct'] = trades['PnL'] / trades['Equity'].shift(1)
        trades.loc[0, 'ReturnPct'] = trades.loc[0, 'PnL'] / INITIAL_CASH
        # --------------------------------------------------------------------------------------------------------------------------------
        
        
        trades['margin'] = (np.abs(trades['Size']) * trades['EntryPrice']) / leverage

        # multiplico el porcentaje de retorno por el nuevo riesgo para obtener los nuevos trades
        syntetic_trades = trades.copy()
        syntetic_trades[['ReturnPct', 'Size']] = syntetic_trades[['ReturnPct', 'Size']] * risk
        syntetic_trades['Equity'] = INITIAL_CASH * (1 + syntetic_trades['ReturnPct']).cumprod()

        syntetic_trades['PnL'] = syntetic_trades['Equity'] - syntetic_trades['Equity'].shift(1)
        syntetic_trades.loc[0, 'PnL'] = syntetic_trades.loc[0, 'Equity'] - INITIAL_CASH
        syntetic_trades['margin'] = (np.abs(syntetic_trades['Size']) * syntetic_trades['EntryPrice']) / leverage
        
        syntetic_trades['Date'] = syntetic_trades['ExitTime']
        syntetic_trades = syntetic_trades.set_index('Date')
    
        original_and_syntetic_trades[strategy][f'{strategy}_r{risk}'] = syntetic_trades
        original_and_syntetic_eq_curves[strategy][f'{strategy}_r{risk}'] = syntetic_trades.Equity

BbandsCross_GBPUSD_16385
no correr GBPUSD con risk 0.25
no correr GBPUSD con risk 0.5
BbandsCross_USDCHF_16386
no correr USDCHF con risk 0.25
no correr USDCHF con risk 0.5
BPercent_EURUSD_16386
no correr EURUSD con risk 0.25
no correr EURUSD con risk 0.5
Channel_NVDA_16387
no correr NVDA con risk 1
no correr NVDA con risk 1.5
no correr NVDA con risk 2
ShortIBS_BABA_16385
no correr BABA con risk 1
no correr BABA con risk 1.5
no correr BABA con risk 2
ShortIBS_EURUSD_16387
no correr EURUSD con risk 0.25
no correr EURUSD con risk 0.5
TripleSMA_NVDA_16386
no correr NVDA con risk 1
no correr NVDA con risk 1.5
no correr NVDA con risk 2


# Combinaciones posibles de curvas con riesgos simulados

In [10]:
import itertools

n_strategies= len(original_and_syntetic_eq_curves.keys())

sets = []
for k, v in original_and_syntetic_eq_curves.items():
    sets.append(list(v.keys()))

# Generar todas las combinaciones posibles tomando al menos 2 conjuntos
combinations = []

# Iterar sobre todos los tamaños posibles (2 o 3 elementos)
for r in range(3, n_strategies + 1): 
    for combination in itertools.combinations(sets, r):
        # Generar el producto cartesiano para la combinación seleccionada
        combinations.extend(itertools.product(*combination))


print(len(combinations))
combinations


6755


[('BbandsCross_GBPUSD_16385_r1',
  'BbandsCross_USDCHF_16386_r1',
  'BPercent_EURUSD_16386_r1'),
 ('BbandsCross_GBPUSD_16385_r1',
  'BbandsCross_USDCHF_16386_r1',
  'BPercent_EURUSD_16386_r1.5'),
 ('BbandsCross_GBPUSD_16385_r1',
  'BbandsCross_USDCHF_16386_r1',
  'BPercent_EURUSD_16386_r2'),
 ('BbandsCross_GBPUSD_16385_r1',
  'BbandsCross_USDCHF_16386_r1.5',
  'BPercent_EURUSD_16386_r1'),
 ('BbandsCross_GBPUSD_16385_r1',
  'BbandsCross_USDCHF_16386_r1.5',
  'BPercent_EURUSD_16386_r1.5'),
 ('BbandsCross_GBPUSD_16385_r1',
  'BbandsCross_USDCHF_16386_r1.5',
  'BPercent_EURUSD_16386_r2'),
 ('BbandsCross_GBPUSD_16385_r1',
  'BbandsCross_USDCHF_16386_r2',
  'BPercent_EURUSD_16386_r1'),
 ('BbandsCross_GBPUSD_16385_r1',
  'BbandsCross_USDCHF_16386_r2',
  'BPercent_EURUSD_16386_r1.5'),
 ('BbandsCross_GBPUSD_16385_r1',
  'BbandsCross_USDCHF_16386_r2',
  'BPercent_EURUSD_16386_r2'),
 ('BbandsCross_GBPUSD_16385_r1.5',
  'BbandsCross_USDCHF_16386_r1',
  'BPercent_EURUSD_16386_r1'),
 ('BbandsCross_G

# Portfolios con riesgo simulado

In [11]:
import re
from sklearn.linear_model import LinearRegression

def safe_mean(arr):
    return np.mean(arr) if len(arr) > 0 else 0

def safe_std(arr):
    return np.std(arr) if len(arr) > 0 else 0

pattern = r"([A-Za-z0-9.]+_[A-Za-z0-9.]+_\d+)_r\d+(\.\d+)?"

results = pd.DataFrame()

for combination in combinations:
    print(combination)
    
    wallet_eq_curves = {}
    wallet_trades = {}
    
    for strategy_name in combination:
        strategy_name_without_risk = re.findall(pattern, strategy_name).pop()[0]
        
        wallet_eq_curves[strategy_name] = original_and_syntetic_eq_curves[strategy_name_without_risk][strategy_name]
        wallet_trades[strategy_name] = original_and_syntetic_trades[strategy_name_without_risk][strategy_name]

    
    combined_equity = get_portfolio_equity_curve(wallet_eq_curves, initial_equity=INITIAL_CASH, date_range=date_range)
    
    # Calcular métricas como retorno y drawdown
    x = np.arange(len(combined_equity)).reshape(-1, 1)
    reg = LinearRegression().fit(x, combined_equity)
    stability_ratio = reg.score(x, combined_equity)

    return_ = ((combined_equity.Equity.iloc[-1] - combined_equity.Equity.iloc[0]) / combined_equity.Equity.iloc[0]) * 100
    dd = np.abs(max_drawdown(combined_equity, verbose=False))

    positive_hits, negative_hits, time_to_positive, time_to_negative = ftmo_simulator(combined_equity, INITIAL_CASH)

    total_hits = positive_hits + negative_hits
    success_ratio = positive_hits / total_hits if total_hits > 0 else 0
    
    all_events, margin_calls, stopouts = calculate_margin_metrics(wallet_trades, combined_equity)

    combination_results = pd.DataFrame({
        'combination': [combination],
        'stability_ratio': [stability_ratio],
        'return': [return_],
        'drawdown': [dd],
        'ret_drawdown': [return_ / dd if dd > 0 else 0],
        'success': [success_ratio],
        'positive_hits': [positive_hits],
        'negative_hits': [negative_hits],
        'mean_time_to_positive': [safe_mean(time_to_positive)],
        'std_time_to_positive': [safe_std(time_to_positive)],
        'mean_time_to_negative': [safe_mean(time_to_negative)],
        'std_time_to_negative': [safe_std(time_to_negative)],
        
        'margin_calls': [margin_calls.shape[0]],
        'stop_outs': [stopouts.shape[0]],
    })


    results = pd.concat([results, combination_results])
    


('BbandsCross_GBPUSD_16385_r1', 'BbandsCross_USDCHF_16386_r1', 'BPercent_EURUSD_16386_r1')
('BbandsCross_GBPUSD_16385_r1', 'BbandsCross_USDCHF_16386_r1', 'BPercent_EURUSD_16386_r1.5')
('BbandsCross_GBPUSD_16385_r1', 'BbandsCross_USDCHF_16386_r1', 'BPercent_EURUSD_16386_r2')
('BbandsCross_GBPUSD_16385_r1', 'BbandsCross_USDCHF_16386_r1.5', 'BPercent_EURUSD_16386_r1')
('BbandsCross_GBPUSD_16385_r1', 'BbandsCross_USDCHF_16386_r1.5', 'BPercent_EURUSD_16386_r1.5')
('BbandsCross_GBPUSD_16385_r1', 'BbandsCross_USDCHF_16386_r1.5', 'BPercent_EURUSD_16386_r2')
('BbandsCross_GBPUSD_16385_r1', 'BbandsCross_USDCHF_16386_r2', 'BPercent_EURUSD_16386_r1')
('BbandsCross_GBPUSD_16385_r1', 'BbandsCross_USDCHF_16386_r2', 'BPercent_EURUSD_16386_r1.5')
('BbandsCross_GBPUSD_16385_r1', 'BbandsCross_USDCHF_16386_r2', 'BPercent_EURUSD_16386_r2')
('BbandsCross_GBPUSD_16385_r1.5', 'BbandsCross_USDCHF_16386_r1', 'BPercent_EURUSD_16386_r1')
('BbandsCross_GBPUSD_16385_r1.5', 'BbandsCross_USDCHF_16386_r1', 'BPercent_E

In [12]:
results.to_csv('./risk_analysis.csv', index=False)

In [13]:
# import pandas as pd
# results = pd.read_csv('./risk_analysis.csv')

In [14]:
filter_results = results[
    (results.margin_calls == 0)
    & (results.negative_hits == 0)
    & (results.drawdown < 10)
].round(1).sort_values(by=['mean_time_to_positive', 'drawdown','ret_drawdown'], ascending=[True, True, False])

filter_results

,combination,stability_ratio,return,drawdown,ret_drawdown,success,positive_hits,negative_hits,mean_time_to_positive,std_time_to_positive,mean_time_to_negative,std_time_to_negative,margin_calls,stop_outs
0,"(BbandsCross_GBPUSD_16385_r2, BbandsCross_USDC...",0.9,521.9,9.8,53.0,1.0,44,0,2.8,1.7,0.0,0.0,0,0
0,"(BbandsCross_GBPUSD_16385_r2, BbandsCross_USDC...",0.9,493.5,9.0,54.7,1.0,44,0,2.9,1.6,0.0,0.0,0,0
0,"(BbandsCross_GBPUSD_16385_r2, BbandsCross_USDC...",0.9,371.5,9.3,40.0,1.0,40,0,2.9,1.5,0.0,0.0,0,0
0,"(BbandsCross_GBPUSD_16385_r2, BbandsCross_USDC...",0.9,350.0,9.5,36.9,1.0,40,0,2.9,1.5,0.0,0.0,0,0
0,"(BbandsCross_GBPUSD_16385_r2, BbandsCross_USDC...",0.9,408.0,9.6,42.4,1.0,40,0,2.9,1.5,0.0,0.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,"(BbandsCross_USDCHF_16386_r1, ShortIBS_BABA_16...",0.9,55.0,5.2,10.6,1.0,35,0,10.8,5.2,0.0,0.0,0,0
0,"(BbandsCross_USDCHF_16386_r1, ShortIBS_EURUSD_...",0.9,47.6,4.4,10.8,1.0,35,0,11.4,5.4,0.0,0.0,0,0
0,"(BbandsCross_USDCHF_16386_r1, ShortIBS_BABA_16...",1.0,44.3,2.5,17.6,1.0,36,0,11.6,3.7,0.0,0.0,0,0
0,"(BbandsCross_USDCHF_16386_r1.5, ShortIBS_BABA_...",1.0,40.7,3.3,12.3,1.0,37,0,11.8,3.3,0.0,0.0,0,0


In [ ]:
0/0

In [15]:
filter_results.iloc[0].combination

('BbandsCross_GBPUSD_16385_r2',
 'BbandsCross_USDCHF_16386_r2',
 'Channel_NVDA_16387_r0.5',
 'ShortIBS_BABA_16385_r0.5',
 'ShortIBS_EURUSD_16387_r1.5',
 'TripleSMA_NVDA_16386_r0.5')

In [16]:
results.to_csv('./risk_analysis.csv', index=False)

In [17]:
original_and_syntetic_eq_curves['BPercent_EURUSD_16386'].keys()

dict_keys(['BPercent_EURUSD_16386_r1', 'BPercent_EURUSD_16386_r1.5', 'BPercent_EURUSD_16386_r2'])

In [18]:
original_and_syntetic_eq_curves['BPercent_EURUSD_16386']['BPercent_EURUSD_16386_r1']

Date
2021-02-09 10:00:00+00:00     9901.332854
2021-02-17 08:00:00+00:00     9801.597968
2021-02-26 18:00:00+00:00     9704.721524
2021-03-11 18:00:00+00:00     9606.647273
2021-03-15 10:00:00+00:00     9720.213719
                                 ...     
2024-10-01 14:00:00+00:00    12525.679381
2024-10-25 20:00:00+00:00    12693.298348
2024-10-29 16:00:00+00:00    12910.753417
2024-10-30 18:00:00+00:00    12780.908852
2024-11-01 00:00:00+00:00    12704.235186
Name: Equity, Length: 166, dtype: float64

# Cálculo del margen total del portfolio

In [19]:
# Curva de equity general calculada con distinto riesgo en cada bot
# Margin acumulado en cada momento del tiempo de esa curva de equity general
# valor del percentil 90 del margin y cantidad de veces que lo supero
# Ver si en mt5 puedo traer el tipo de activo que es cada ticker, para ver cuanto leverage ponerle
# Ver si tengo que reescalar algo de los precios para calcular el margin
# agregar un parametro de risk por cada combinacion de activo-bot para permitir que corran con diferentes riesgos
# Columna con cantidad de margin calls y step outs

In [20]:



if 'variaciones_porcentuales' in all_equity_curves.keys():
    del all_equity_curves['variaciones_porcentuales']

total = get_portfolio_equity_curve(equity_curves=all_equity_curves, initial_equity=INITIAL_CASH, date_range=date_range)
all_equity_curves['variaciones_porcentuales'] = total

In [21]:
variaciones_porcentuales = all_equity_curves['variaciones_porcentuales']
variaciones_porcentuales.index = pd.to_datetime(variaciones_porcentuales.index)

In [22]:
stopouts.shape[0]

0

In [23]:
# all_events.to_csv("./margins.csv",index=False)

In [24]:
import plotly.graph_objects as go

# Crear una figura vacía
fig = go.Figure()

# Recorrer las curvas de equity de cada bot y agregarlas al gráfico
fig.add_trace(go.Scatter(x=all_events.index, y=all_events.Equity, mode='lines', name='Equity'))
fig.add_trace(go.Scatter(x=stop_out.index, y=stop_out.Equity, mode='markers', name='Stop Out'))
fig.add_trace(go.Scatter(x=margin_calls.index, y=margin_calls.Equity, mode='markers', name='Margin calls'))
fig.add_trace(go.Scatter(x=all_events.index, y=all_events.margin, mode='lines', name='margin'))
fig.add_trace(go.Scatter(x=all_events.index, y=all_events.margin_level, mode='lines', name='margin_level'))
fig.add_trace(go.Scatter(x=all_events.index, y=all_events.free_margin, mode='lines', name='free_margin'))

# Agregar una línea horizontal en y=100
fig.add_shape(
    type="line",
    x0=all_events.index.min(),
    x1=all_events.index.max(),
    y0=100,
    y1=100,
    line=dict(color="red", width=2),  # Color rojo, línea discontinua
    name="y=100"
)

# Actualizar los detalles del layout del gráfico
fig.update_layout(
    title="Curva de Margin de Múltiples Bots",
    xaxis_title="Fecha",
    yaxis_title="Equity",
    legend_title="Bots",
    shapes=[  # Asegurar que la línea se incluya en el layout
        dict(
            type="line",
            x0=all_events.index.min(),
            x1=all_events.index.max(),
            y0=100,
            y1=100,
            line=dict(color="red", width=2, dash="dash")
        )
    ]
)

# Mostrar el gráfico
fig.show()

NameError: name 'stop_out' is not defined

In [ ]:
0/0

In [ ]:
import plotly.graph_objects as go

# Crear una figura vacía
fig = go.Figure()

# Recorrer las curvas de equity de cada bot y agregarlas al gráfico
for k, v in all_equity_curves.items():
    
    fig.add_trace(go.Scatter(x=v.index, y=v.Equity, mode='lines', name=k))

# Actualizar los detalles del layout del gráfico
fig.update_layout(
    title="Curvas de Equity de Múltiples Bots",
    xaxis_title="Fecha",
    yaxis_title="Equity",
    legend_title="Bots"
)

# Mostrar el gráfico
fig.show()

max_drawdown(all_equity_curves['variaciones_porcentuales'])


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px

# Supongamos que tienes el DataFrame `df` como en el ejemplo
# Convertir el índice a tipo datetime si no lo está
df = all_equity_curves["variaciones_porcentuales"]
df.index = pd.to_datetime(df.index)

# Calcular los retornos diarios en porcentaje
df['Daily Return'] = ((df['Equity'] - df['Equity'].shift(1)) / df['Equity'].shift(1)) * 100

# Crear un DataFrame resampleado con valores mínimo y máximo para cada mes
monthly_min = df['Daily Return'].resample("M").min()
monthly_max = df['Daily Return'].resample("M").max()

# Aplicar la lógica de np.where para seleccionar el mínimo si < 0, máximo si >= 0
monthly_returns = np.where(monthly_max >= 0, monthly_max, monthly_min)

# Crear un índice temporal basado en las fechas de los datos mensuales
monthly_index = df['Daily Return'].resample("M").apply(lambda x: x.index[-1])

# Crear un DataFrame para el gráfico
monthly_df = pd.DataFrame({
    'Fecha': monthly_index,
    'Retorno Mensual': monthly_returns
})

display(monthly_df)

# Crear el gráfico de barras con Plotly
fig = px.bar(
    monthly_df,
    x='Fecha',
    y='Retorno Mensual',
    labels={"Fecha": "Fecha", "Retorno Mensual": "Retorno Mensual (%)"},
    title="Retornos Mensuales Ajustados en Porcentaje",
    text_auto='.2s'
)

fig.update_layout(
    # width=800,
    # height=700,
    xaxis_title="Fecha",
    yaxis_title="Retorno Mensual (%)",
    xaxis=dict(tickformat="%Y-%m"),
)

fig.show()


In [11]:
del all_equity_curves['variaciones_porcentuales']

In [13]:
# ftmo_simulator(equity_curves['variaciones_porcentuales'], 10_000)

In [ ]:
from itertools import combinations

# Generar todas las combinaciones de las claves del diccionario
keys = list(all_equity_curves.keys())

equity_curves_combinations = []

# Obtenemos combinaciones desde tamaño 1 hasta len(keys)
for r in range(2, len(keys) + 1):
    for combination in combinations(keys, r):
        equity_curves_combinations.append(combination)

equity_curves_combinations

In [ ]:

from sklearn.linear_model import LinearRegression

stability_ratio = 0
best_ratio = 0

results = pd.DataFrame()

for combination in equity_curves_combinations:
    selected_curves = {}
    for name in combination:
        selected_curves[name] = all_equity_curves[name]
    
    # Generar la curva combinada
    combined_equity = get_portfolio_equity_curve(selected_curves, initial_equity=INITIAL_CASH)
    
    # Calcular métricas como retorno y drawdown
    x = np.arange(len(combined_equity)).reshape(-1, 1)
    reg = LinearRegression().fit(x, combined_equity)
    stability_ratio = reg.score(x, combined_equity)
    
    return_ = ((combined_equity.Equity.iloc[-1] - combined_equity.Equity.iloc[0]) / combined_equity.Equity.iloc[0]) * 100
    dd = -1 * max_drawdown(combined_equity, verbose=False)
    
    positive_hits, negative_hits, time_to_positive, time_to_negative = ftmo_simulator(combined_equity, INITIAL_CASH)

    combination_results = pd.DataFrame({
        'combination':[combination],
        'stability_ratio':[stability_ratio],
        'return':[return_],
        'drawdown':[dd],
        'ret_drawdown':[return_/dd],
        
        'success':[positive_hits / (positive_hits + negative_hits)],
        'positive_hits':[positive_hits],
        'negative_hits':[negative_hits],
        'mean_time_to_positive':[np.mean(time_to_positive)],
        'std_time_to_positive':[np.std(time_to_positive)],
        'mean_time_to_negative':[np.mean(time_to_negative)],
        'std_time_to_negative':[np.std(time_to_negative)],
        
    })

    results = pd.concat([results, combination_results])
    
results.sort_values(by='ret_drawdown', ascending=False)

In [15]:
# results.to_csv('./wallet_results.csv')

In [ ]:

0/0

In [16]:
# ShortIBS_ADAUSD_H4 = equity_curves['ShortIBS_ADAUSD_H4']
# ShortIBS_DOTUSD_H4 = equity_curves['ShortIBS_DOTUSD_H4']
# TripleST_NVDA_H4 = equity_curves['TripleST_NVDA_H4']
# MeanRev_XMRUSD_H4 = equity_curves['MeanRev_XMRUSD_H1']

# del equity_curves['TripleST_NVDA_H4']
# del equity_curves['TripleSMA_US100.cash_H2']
# del equity_curves['TripleST_TSLA_H3']


# del equity_curves['MeanRev_XMRUSD_H1']


# equity_curves['ShortIBS_ADAUSD_H4'] = ShortIBS_ADAUSD_H4
# equity_curves['ShortIBS_DOTUSD_H4'] = ShortIBS_DOTUSD_H4
# equity_curves['TripleST_NVDA_H4'] = TripleST_NVDA_H4
# equity_curves['MeanRev_XMRUSD_H4'] = MeanRev_XMRUSD_H4

# Correlaciones

In [ ]:
percentual_differences = {}

def get_percentual_differences(equity_curves):
    for name, curve in equity_curves.items():
        equity_df = curve.copy()
        
        equity_df = equity_df.reset_index().rename(columns={'index':'Date'})
        equity_df['Date'] = pd.to_datetime(equity_df['Date'])
        equity_df['Date'] = equity_df['Date'].dt.floor('D').dt.date
        
        equity_df = pd.DataFrame(equity_df.groupby('Date')['Equity'].last()).reindex(date_range)
        equity_df.ffill(inplace=True)
        equity_df.fillna(INITIAL_CASH, inplace=True)

        equity_df['diff'] = equity_df['Equity'] - equity_df['Equity'].shift(1)
        
        percentual_differences[name] = equity_df
    
    return percentual_differences


differences = get_percentual_differences(all_equity_curves)
differences

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Paso 1: Unir todas las curvas de equity en un solo DataFrame basado en la fecha
all_equity_df = pd.DataFrame()

for name, df in differences.items():
    all_equity_df[name] = df.resample('M').agg({'Equity':'last','diff':'sum',})['diff']


# Paso 2: Calcular la matriz de correlación

# all_equity_df = all_equity_df[['BPercent_IQm_H1','BPercent_NTESm_H1']]
correlation_matrix = all_equity_df.corr(method='pearson')

# Paso 3: Plotear el mapa de calor de correlación usando seaborn
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Equity Curves')
plt.xticks(rotation=75)
plt.show()

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Paso 1: Unir todas las curvas de equity en un solo DataFrame basado en la fecha
all_equity_df = pd.DataFrame()

for name, df in differences.items():
    all_equity_df[name] = df.resample('W').agg({'Equity':'last','diff':'sum',})['diff']


# Paso 2: Calcular la matriz de correlación

# all_equity_df = all_equity_df[['BPercent_IQm_H1','BPercent_NTESm_H1']]
correlation_matrix = all_equity_df.corr(method='pearson')

# Paso 3: Plotear el mapa de calor de correlación usando seaborn
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Matrix of Equity Curves')
plt.xticks(rotation=75)
plt.show()